In [28]:
import pandas as pd
import numpy as np

In [29]:
df = pd.read_csv("data/exp_transfer_learning.csv")

In [30]:
df = df.dropna(subset=['text'])

In [31]:
import re
# Step 3: Remove Special Characters
def clean_text(text):
    # Remove special characters (you can customize the regex as needed)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # You can also remove extra spaces or normalize spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the clean_text function to the 'text' column
df['text'] = df['text'].apply(clean_text)

In [32]:
df.head(5)

,dataset_name,text,label_name,label_value,label_definition,source,language
0,OLID19,USER She should ask a few native Americans wha...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
1,OLID19,USER USER Go home youre drunk USER MAGA Trump2...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
2,OLID19,Amazon is investigating Chinese employees who ...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
3,OLID19,USER Someone shouldveTaken this piece of shit ...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
4,OLID19,USER USER Obama wanted liberals amp illegals t...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng


In [33]:
df_target = df[df["dataset_name"] == "EDOS"].copy()
df_target

,dataset_name,text,label_name,label_value,label_definition,source,language


In [34]:
df_target['label_value'] = df_target.apply(lambda row: 0 if row["label_value"] == "not sexist" else 1, axis=1)
df_target

ValueError: Columns must be same length as key

In [35]:
df_aux = df[df["dataset_name"] != "EDOS"].copy()
df_aux

,dataset_name,text,label_name,label_value,label_definition,source,language
0,OLID19,USER She should ask a few native Americans wha...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
1,OLID19,USER USER Go home youre drunk USER MAGA Trump2...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
2,OLID19,Amazon is investigating Chinese employees who ...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
3,OLID19,USER Someone shouldveTaken this piece of shit ...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng
4,OLID19,USER USER Obama wanted liberals amp illegals t...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
...,...,...,...,...,...,...,...
4199,OLID19,AfD Like a BOSS AfDOrdner reagiert COOL Antifa...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
4200,OLID19,98 of all mass shootings occur in gun free zon...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
4201,OLID19,USER USER USER USER USER USER Conservatives wh...,subtask_a,NOT,Level A: Offensive language identification \r\...,Twitter,eng
4202,OLID19,USER The washedout idiot Biden feels he has a ...,subtask_a,OFF,Level A: Offensive language identification \r\...,Twitter,eng


In [26]:
unique_values = df_aux.groupby("dataset_name")['label_value'].apply(lambda x: list(x.unique()))
unique_values

dataset_name
OLID19    [OFF, NOT, nan]
Name: label_value, dtype: object

In [27]:
def map_label_value(row):
    if row["dataset_name"] == "DGHD":
        if row["label_value"] == "hate":
            return 1
    
    elif row["dataset_name"] == "OLID":
        if row["label_value"] == "OFF":
            return 1
    
    elif row["dataset_name"] == "MHS":
        if float(row["label_value"]) > 0.5:
            return 1
    
    elif row["dataset_name"] == "Ethos":
        if float(row["label_value"]) >= 0.5:
            return 1
    
    return 0

In [13]:
df_aux["label_value"] = df.apply(map_label_value,axis=1)

NameError: name 'df_aux' is not defined

In [14]:
df_aux

NameError: name 'df_aux' is not defined

In [10]:
len(df_aux[df_aux["label_value"]==1]) / len(df_aux)

NameError: name 'df_aux' is not defined

In [11]:
len(df_target[df_target["label_value"]==1]) / len(df_target)

ZeroDivisionError: division by zero

In [20]:
from Sampler import balanced_fixedcount

aux_train = balanced_fixedcount(df_aux, 100000, "label_value", "dataset_name")
aux_train

   label second_col
0      A          Y
1      A          Z
2      A          X
3      A          Y
4      A          Y
5      B          Z
6      B          X
7      B          Z
8      B          Z
9      B          X
10     C          Z
11     C          Z
12     C          Y
13     C          Z
14     C          Z
  label second_col
0     A          X
1     A          Z
2     A          X
3     B          X
4     B          Y
5     B          Z
6     C          Y
7     C          Z
8     C          Y
   label second_col
0      C          Y
1      C          Z
2      A          Y
3      C          Y
4      B          X
5      A          X
6      A          Z
7      C          X
8      B          Z
9      B          Z
10     A          Y
11     B          Z
   label second_col
0      A          X
1      A          X
2      A          Y
3      A          Y
4      A          Z
5      A          Z
6      B          X
7      B          X
8      B          Y
9      B          Y
10     B  

,dataset_name,text,label_name,label_value,label_definition,source,language
0,DGHD,Young Womens Trust needs to be destroyed or th...,label,0,"a binary variable, indicating whether or not t...",NaN,eng
1,MHS,DoingRight1 Why did anyone think allowing hord...,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
2,MHS,Life without Allah is meaningless,hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
3,DGHD,this is a Fucken blaady madlad,label,0,"a binary variable, indicating whether or not t...",NaN,eng
4,MHS,Okay rniceguys front man Whos the incel again,hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
...,...,...,...,...,...,...,...
99995,MHS,libertyladyusa Deport this trash,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99996,MHS,Ha Take that nigger,hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99997,MHS,Hitler didnt finish it Can u If a nigger ur Je...,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99998,Ethos,When lift your arms do your boobs go away They...,isHate,0,NaN,Multi,eng


In [21]:
len(aux_train[aux_train["label_value"]==1]) / len(aux_train)

0.45541

## Target Split

In [22]:
from sklearn.model_selection import train_test_split
train_tdf, test_tdf = train_test_split(df_target, test_size=0.2, stratify=df_target['label_value'], random_state=42)

In [23]:
from tool.utils.sampling import balanced_downsampling
print("Before Downsampling: ",len(test_tdf[test_tdf["label_value"]==1]) / len(test_tdf))

resampled_test_tdf = balanced_downsampling(test_tdf, "label_value")

print("After Downsampling: ",len(resampled_test_tdf[resampled_test_tdf["label_value"]==1]) / len(resampled_test_tdf))

Before Downsampling:  0.24275
After Downsampling:  0.5


In [24]:
train_tdf.to_csv("train_tdf.csv")
test_tdf.to_csv("test_tdf.csv")
aux_train.to_csv("aux_train.csv")

In [25]:
for train_per in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    _, train_tdf_slice = train_test_split(train_tdf, test_size = train_per, stratify=train_tdf['label_value'], random_state=42)

ValueError: test_size=1.0 should be either positive and smaller than the number of samples 16000 or a float in the (0, 1) range